# Matrix-matrix multiplication via rank-1 updates

We continue to look at how the FLAMEPy API can be used to implement different matrix-matrix multiplication algorithms.  

First, we create some matrices.

In [1]:
import numpy as np

m = 4
n = 3
k = 5

C = np.matrix( np.random.random( (m, n) ) )
print( 'C = ' )
print( C )

Cold = np.matrix( np.zeros( (m,n ) ) )
Cold = np.matrix( np.copy( C ) )           # an alternative way of doing a "hard" copy, in this case of a matrix
    
A = np.matrix( np.random.random( (m, k) ) )
print( 'A = ' )
print( A )

B = np.matrix( np.random.random( (k, n) ) )
print( 'B = ' )
print( B )

C = 
[[0.73234942 0.95654744 0.73065344]
 [0.06058527 0.26518135 0.05949556]
 [0.26368915 0.64480221 0.39025924]
 [0.18527076 0.79065465 0.69981963]]
A = 
[[0.3456013  0.8246311  0.0938523  0.81712455 0.07526462]
 [0.67799278 0.34871588 0.75069561 0.1232985  0.96173853]
 [0.44541298 0.84094292 0.46018034 0.76823269 0.92676968]
 [0.89349538 0.64991332 0.33405298 0.24503337 0.46392869]]
B = 
[[0.14514004 0.03103341 0.57079897]
 [0.97667934 0.28860119 0.33909399]
 [0.30569733 0.34647187 0.97702337]
 [0.6783205  0.53637549 0.42226538]
 [0.57710241 0.00750559 0.63989082]]


## <h2>The algorithm  </h2>  <image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/Gemm_nn_unb_var3.png" alt="Matrix-matrix multiplication via rank-1 updates picture" width="80%">

<h2> The routine <code> Gemm_nn_unb_var3( A, B, C ) </code> </h2>

This routine computes $ C := A B + C $ via rank-1 updates.  The "\_nn\_" means that this is the "No transpose, No transpose" case of matrix multiplication.  
The reason for this is that the operations $ C := A^T B + C $ ("\_tn\_" or "Transpose, No transpose"), $ C := A B^T + C $ ("\_nt\_" or "No transpose, Transpose"), and $ C := A^T B^T + C $ ("\_tt\_" or "Transpose, Transpose") are also encountered.  
    
The specific laff function we will use is
<ul>
<li> <code> laff.ger( alpha, x, y, A ) </code> which computes the rank-1 update (adds a multiple of an outer product to a matrix)
$ A := \alpha x y^T + A $. 
    </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [2]:
import flame
import laff as laff

def Gemm_nn_unb_var3(A, B, C):

    AL, AR = flame.part_1x2(A, \
                            0, 'LEFT')

    BT, \
    BB  = flame.part_2x1(B, \
                         0, 'TOP')

    while AL.shape[1] < A.shape[1]:

        A0, a1, A2 = flame.repart_1x2_to_1x3(AL, AR, \
                                             1, 'RIGHT')

        B0,  \
        b1t, \
        B2   = flame.repart_2x1_to_3x1(BT, \
                                       BB, \
                                       1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.ger( 1.0, a1, b1t, C )

        #------------------------------------------------------------#

        AL, AR = flame.cont_with_1x3_to_1x2(A0, a1, A2, \
                                            'LEFT')

        BT, \
        BB  = flame.cont_with_3x1_to_2x1(B0,  \
                                         b1t, \
                                         B2,  \
                                         'TOP')



In [3]:
C = np.matrix( np.copy( Cold ) )               # restore C 

Gemm_nn_unb_var3( A, B, C )

print( 'C - ( Cold + A * B )' )
print( C - ( Cold + A * B ) )

C - ( Cold + A * B )
[[-4.44089210e-16 -2.22044605e-16  2.22044605e-16]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [-4.44089210e-16  2.22044605e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


Bingo! It works!

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.